# Laborator 4: Machine Learning Clasic pentru Detectarea Intruziunilor

## Obiective
- Antrenarea modelelor de clasificare clasice
- Compararea performanțelor între algoritmi
- Înțelegerea hiperparametrilor principali

## Algoritmi implementați
1. **Decision Tree** - Arbore de decizie
2. **Random Forest** - Pădure de arbori (ensemble)
3. **K-Nearest Neighbors (KNN)** - Bazat pe distanță

## Prerequisite
Acest laborator folosește datele preprocesate din Laboratorul 3.

## 1. Setup și Import

In [ ]:
# Instalare dependențe (doar în Colab)
# !pip install pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.model_selection import cross_val_score, GridSearchCV

plt.style.use('seaborn-v0_8-whitegrid')
print("Biblioteci încărcate cu succes!")

## 2. Încărcarea Datelor Preprocesate

In [ ]:
# Încărcăm datele din Lab 3
# Dacă nu există, creăm date sintetice

try:
    X_train = np.load('X_train.npy')
    X_test = np.load('X_test.npy')
    y_train = np.load('y_train.npy')
    y_test = np.load('y_test.npy')
    print("Date încărcate din fișierele salvate!")
except FileNotFoundError:
    print("Fișierele nu au fost găsite. Creăm date sintetice...")
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    X, y = make_classification(
        n_samples=10000, n_features=38, n_informative=20,
        n_redundant=10, n_classes=2, weights=[0.5, 0.5],
        random_state=42
    )
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("Date sintetice create!")

print(f"\nTrain set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Clase: 0={sum(y_train==0)}, 1={sum(y_train==1)}")

## 3. Funcții Helper pentru Evaluare

In [ ]:
def evaluate_model(model, X_test, y_test, model_name="Model"):
    """Evaluează un model și afișează metricile."""
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"\n{'='*50}")
    print(f"Rezultate {model_name}")
    print(f"{'='*50}")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    """Afișează matricea de confuzie."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Attack'],
                yticklabels=['Normal', 'Attack'])
    plt.title(title, fontsize=14)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# Dicționar pentru stocarea rezultatelor
results = []

## 4. Decision Tree (Arbore de Decizie)

### Cum funcționează?
- Împarte datele recursiv pe baza unor condiții (ex: `feature_5 > 0.5`)
- La fiecare nod, alege split-ul care maximizează "puritatea" (Gini sau Entropy)
- Frunzele conțin predicția finală

### Hiperparametri importanți:
- `max_depth`: Adâncimea maximă a arborelui
- `min_samples_split`: Numărul minim de samples pentru a face un split
- `criterion`: 'gini' sau 'entropy'

In [ ]:
# Antrenăm un Decision Tree simplu
print("Antrenare Decision Tree...")

start_time = time.time()

dt_model = DecisionTreeClassifier(
    max_depth=10,           # Limităm adâncimea pentru a evita overfitting
    min_samples_split=10,   # Minim 10 samples pentru split
    criterion='gini',       # Criteriu de split
    random_state=42
)

dt_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Timp antrenare: {train_time:.2f}s")

In [ ]:
# Evaluăm modelul
dt_results = evaluate_model(dt_model, X_test, y_test, "Decision Tree")
results.append(dt_results)

In [ ]:
# Matricea de confuzie
y_pred_dt = dt_model.predict(X_test)
plot_confusion_matrix(y_test, y_pred_dt, "Decision Tree - Confusion Matrix")

In [ ]:
# Vizualizare arbore (primele nivele)
plt.figure(figsize=(20, 10))
plot_tree(dt_model, max_depth=3, filled=True, fontsize=10,
          class_names=['Normal', 'Attack'])
plt.title("Decision Tree (primele 3 nivele)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': [f'feature_{i}' for i in range(X_train.shape[1])],
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
plt.bar(range(15), feature_importance['importance'].head(15))
plt.xticks(range(15), feature_importance['feature'].head(15), rotation=45)
plt.title('Top 15 Feature Importance - Decision Tree')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

## 5. Random Forest (Pădure Aleatoare)

### Cum funcționează?
- Antrenează mai mulți arbori de decizie pe subset-uri diferite de date (bagging)
- Fiecare arbore folosește un subset random de features
- Predicția finală = votul majoritar al tuturor arborilor

### De ce e mai bun?
- Reduce overfitting prin averaging
- Mai robust la zgomot
- Paralelizabil

### Hiperparametri importanți:
- `n_estimators`: Numărul de arbori
- `max_depth`: Adâncimea maximă per arbore
- `max_features`: Numărul de features considerate la fiecare split

In [ ]:
# Antrenăm Random Forest
print("Antrenare Random Forest...")

start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=100,       # 100 de arbori
    max_depth=15,           # Adâncime maximă
    min_samples_split=5,    # Minim 5 samples pentru split
    max_features='sqrt',    # sqrt(n_features) pentru fiecare split
    n_jobs=-1,              # Folosește toate CPU-urile
    random_state=42
)

rf_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Timp antrenare: {train_time:.2f}s")

In [ ]:
# Evaluăm modelul
rf_results = evaluate_model(rf_model, X_test, y_test, "Random Forest")
results.append(rf_results)

In [ ]:
# Matricea de confuzie
y_pred_rf = rf_model.predict(X_test)
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest - Confusion Matrix")

In [ ]:
# Feature importance pentru Random Forest
rf_feature_importance = pd.DataFrame({
    'feature': [f'feature_{i}' for i in range(X_train.shape[1])],
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
plt.bar(range(15), rf_feature_importance['importance'].head(15), color='forestgreen')
plt.xticks(range(15), rf_feature_importance['feature'].head(15), rotation=45)
plt.title('Top 15 Feature Importance - Random Forest')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

## 6. K-Nearest Neighbors (KNN)

### Cum funcționează?
- Pentru o nouă instanță, găsește cele mai apropiate K puncte din training set
- Clasifică pe baza votului majoritar al vecinilor
- "Lazy learner" - nu antrenează un model explicit

### Hiperparametri importanți:
- `n_neighbors` (K): Numărul de vecini
- `metric`: Metrica de distanță (euclidean, manhattan)
- `weights`: 'uniform' sau 'distance' (vecinii mai apropiați au greutate mai mare)

### Dezavantaje:
- Lent la predicție (calculează distanțe față de toate punctele)
- Sensibil la "curse of dimensionality"

In [ ]:
# Antrenăm KNN
print("Antrenare K-Nearest Neighbors...")

start_time = time.time()

knn_model = KNeighborsClassifier(
    n_neighbors=5,          # K=5 vecini
    metric='euclidean',     # Distanță euclidiană
    weights='distance',     # Vecinii apropiați au greutate mai mare
    n_jobs=-1               # Paralelizare
)

knn_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Timp antrenare: {train_time:.2f}s")

In [ ]:
# Evaluăm modelul
knn_results = evaluate_model(knn_model, X_test, y_test, "KNN (K=5)")
results.append(knn_results)

In [ ]:
# Matricea de confuzie
y_pred_knn = knn_model.predict(X_test)
plot_confusion_matrix(y_test, y_pred_knn, "KNN - Confusion Matrix")

In [ ]:
# Experimentăm cu diferite valori de K
k_values = [1, 3, 5, 7, 9, 11, 15, 21]
k_scores = []

print("Testare diferite valori de K...")
for k in k_values:
    knn_temp = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn_temp.fit(X_train, y_train)
    score = knn_temp.score(X_test, y_test)
    k_scores.append(score)
    print(f"K={k}: Accuracy={score:.4f}")

# Vizualizare
plt.figure(figsize=(10, 5))
plt.plot(k_values, k_scores, 'bo-', linewidth=2, markersize=8)
plt.xlabel('K (Număr de vecini)')
plt.ylabel('Accuracy')
plt.title('Performanța KNN în funcție de K')
plt.xticks(k_values)
plt.grid(True)
plt.show()

## 7. Compararea Modelelor

In [ ]:
# Afișăm rezultatele comparative
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("COMPARAȚIE MODELE")
print("="*60)
print(results_df.to_string(index=False))

In [ ]:
# Vizualizare comparativă
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Grafic 1: Toate metricile
metrics = ['accuracy', 'precision', 'recall', 'f1']
x = np.arange(len(results_df))
width = 0.2

for i, metric in enumerate(metrics):
    axes[0].bar(x + i*width, results_df[metric], width, label=metric.capitalize())

axes[0].set_xlabel('Model')
axes[0].set_ylabel('Scor')
axes[0].set_title('Comparație Metrici pe Modele')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(results_df['model'])
axes[0].legend()
axes[0].set_ylim([0.8, 1.0])

# Grafic 2: F1-Score (cea mai importantă metrică pentru clasificare dezechilibrată)
colors = ['steelblue', 'forestgreen', 'coral']
axes[1].bar(results_df['model'], results_df['f1'], color=colors)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('F1-Score')
axes[1].set_title('F1-Score per Model')
axes[1].set_ylim([0.8, 1.0])

for i, v in enumerate(results_df['f1']):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.show()

## 8. Cross-Validation

In [ ]:
# Cross-validation pentru o evaluare mai robustă
print("Cross-validation (5-fold)...\n")

models = {
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
}

cv_results = []
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    print(f"{name}:")
    print(f"  F1 scores: {scores}")
    print(f"  Mean: {scores.mean():.4f} (+/- {scores.std()*2:.4f})")
    cv_results.append({'model': name, 'mean_f1': scores.mean(), 'std_f1': scores.std()})
    print()

## 9. Salvarea Modelelor

In [ ]:
# Salvăm modelele pentru Lab 6
models_to_save = {
    'decision_tree': dt_model,
    'random_forest': rf_model,
    'knn': knn_model
}

for name, model in models_to_save.items():
    with open(f'{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print(f"Salvat: {name}_model.pkl")

# Salvăm și rezultatele
results_df.to_csv('classical_ml_results.csv', index=False)
print("\nRezultate salvate în: classical_ml_results.csv")

## 10. Rezumat și Concluzii

### Ce am învățat:
1. **Decision Tree** - Simplu, interpretabil, dar tinde să facă overfitting
2. **Random Forest** - Mai robust, accuracy mai bună, dar mai lent
3. **KNN** - Simplu conceptual, dar lent la predicție pe date mari

### Recomandări:
- Pentru interpretabilitate: Decision Tree
- Pentru performanță: Random Forest
- Pentru prototipare rapidă: KNN

### Următorul pas:
**Laborator 5** - Deep Learning cu rețele neurale (MLP, LSTM)

In [ ]:
# Sumar final
print("\n" + "="*60)
print("SUMAR FINAL - LABORATOR 4")
print("="*60)
print(f"\nCel mai bun model (F1-Score): {results_df.loc[results_df['f1'].idxmax(), 'model']}")
print(f"F1-Score maxim: {results_df['f1'].max():.4f}")
print(f"\nFișiere salvate:")
print("  - decision_tree_model.pkl")
print("  - random_forest_model.pkl")
print("  - knn_model.pkl")
print("  - classical_ml_results.csv")